In [ ]:
%%javascript

import Root from '../../../../../notebooks/treezjs/src/root/root.js';
import ColumnType from '../../../../../notebooks/treezjs/src/data/column/columnType.js';
import BorderMode from '../../../../../notebooks/treezjs/src/result/axis/borderMode.js';
import AxisMode from '../../../../../notebooks/treezjs/src/result/axis/axisMode.js';
import Direction from '../../../../../notebooks/treezjs/src/result/axis/direction.js';

window.createModel = function(){

    let root = new Root();

    //#region MODELS ------------------------------------------------------
    let models = root.createModels();
    models._studyId = 'mystudyId';


    //#region path

    let path = models.createPath();
    path._studyId = 'mystudyId';

    path.createDirectoryPathVariable('workingDir', '../..');

    //#end region

    //#region genericInput

    let genericInput = models.createGenericInput();
    genericInput._studyId = 'mystudyId';

    genericInput.createIntegerVariable('x', 10);
    genericInput.createIntegerVariable('y', 100);

    //#end region

    //#region inputFileGenerator

    let inputFileGenerator = models.createInputFileGenerator();
    inputFileGenerator.templatePath = '{$workingDir$}/template.txt';
    inputFileGenerator.valueExpression = '<value>';
    inputFileGenerator.inputFilePath = '{$workingDir$}/input.txt';
    inputFileGenerator._studyId = 'mystudyId';

    //#end region

    //#region executable

    let executable = models.createExecutable();
    executable.executablePath = '{$workingDir$}/executable.bat';
    executable.inputPath = '{$workingDir$}/input.txt';
    executable.outputPath = '{$workingDir$}/output.txt';
    executable._studyId = 'mystudyId';

    //#end region

    //#region tableImport

    let tableImport = models.createTableImport();
    tableImport.filePath = '{$workingDir$}/output.txt';

    //#end region

    //#end region ---------------------------------------------------------

    //#region STUDIES -----------------------------------------------------
    let studies = root.createStudies();

    //#region sensitivity

    let sensitivity = studies.createSensitivity();
    sensitivity.id = 'myStudyId';
    sensitivity.controlledModelPath = 'root.models';
    sensitivity.sourceModelPath = 'root.models.genericInput';
    sensitivity.variableNames = ['x', 'y'];

    //#end region

    //#end region ---------------------------------------------------------

    //#region RESULTS -----------------------------------------------------
    let results = root.createResults();

    //#region data

    let data = results.createData();
    let sensitvityProbe = data.createSensitivityProbe('sensitvityProbe');
    sensitvityProbe.probeLabel = 'z';

    let sensitvityProbeTable = sensitvityProbe.createTable('sensitvityProbeTable');
    let columns = sensitvityProbeTable.createColumnFolder('columns');
    columns.createColumn('x', ColumnType.integer);
    columns.createColumn('y', ColumnType.integer);
    columns.createColumn('z');

    sensitvityProbeTable.createRow([10, 100, "110"]);
    sensitvityProbeTable.createRow([9, 100, "109"]);
    sensitvityProbeTable.createRow([11, 100, "111"]);
    sensitvityProbeTable.createRow([10, 90, "100"]);
    sensitvityProbeTable.createRow([10, 110, "120"]);

    let sensitivityOutput = data.createSensitivityOutput();
    

    //#end region

    //#region page

    let page = results.createPage();
    let graph = page.createGraph();
    let x = graph.createAxis('x');
    x.data.borderMin = BorderMode.ten;
    x.data.borderMax = BorderMode.ten;

    x.tickLabels.tickLabelHeight = 25;

    let y = graph.createAxis('y');
    y.data.mode = AxisMode.ordinal;
    y.data.direction = Direction.vertical;

    y.tickLabels.tickLabelWidth = 12;

    let tornado = graph.createTornado();
    tornado.data.tablePath = 'root.results.data.sensitvityProbe.sensitvityProbeTable';
    tornado.data.inputBase = null;
    tornado.data.inputLabel = null;
    tornado.data.inputLeft = null;
    tornado.data.inputRight = null;
    tornado.data.inputUnit = null;
    tornado.data.outputBase = null;
    tornado.data.outputLeft = null;
    tornado.data.outputRight = null;

    //#end region

    //#end region ---------------------------------------------------------

    return root;
};